In [1]:
# import sys
# !{sys.executable} -m pip install seaborn
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import glacierml as gl
import numpy as np
import warnings
from tensorflow.python.util import deprecation
import os
import logging
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML
import geopy
# display(HTML("<style>.container { width:85% !important; }</style>"))
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.set_option('display.max_columns', None)

pd.set_option('mode.chained_assignment', None)

RS = range(0,25,1)


# df = df[[
#     'area_g',
#     'Area'
# ]]


In [3]:
# select dataset to analyze

selected_dataset = 'df2'
rootdir = '/home/simonhans/data/prethicktor/'

if selected_dataset == 'df1':
    df1 = gl.data_loader(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'n'
    )
    module = 'sm1'
    res = 'sr1'
    dataset = df1
    dataset.name = selected_dataset

if selected_dataset == 'df2':
    df2 = gl.data_loader(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'off'
    )
    module = 'sm2'
    res = 'sr2'
    dataset = df2
    dataset.name = selected_dataset
    
if selected_dataset == 'df3':
    df3 = gl.data_loader(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 1
    )
    module = 'sm3'
    res = 'sr3'
    dataset = df3
    dataset.name = selected_dataset
    
if selected_dataset == 'df4':
    df4 = gl.data_loader(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 5
    )
    module = 'sm4'
    res = 'sr4'
    dataset = df4
    dataset.name = selected_dataset
    
if selected_dataset == 'df5':
    df5 = gl.data_loader(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'y',
        scale = 'r',
        region_selection = 1,
        area_scrubber = 'off'
    )
    module = 'sm5'
    res = 'sr5'
    reg = df5['region'].iloc[-1]
    df5 = df5.drop('region', axis=1)
    dataset = df5
    dataset.name = str('df5_' + str(reg))

if selected_dataset == 'df6':
    df6 = gl.data_loader_6(
        pth_1 = rootdir + 'T_data/',
        pth_2 = rootdir + 'RGI/rgi60-attribs/',
        pth_3 = rootdir + 'matched_indexes/',
        pth_4 = rootdir + 'regional_data/training_data/',
        RGI_input = 'y',
        scale = 'r',
        region_selection = 1,
        area_scrubber = 'off'
    )
    module = 'sm6'
    res = 'sr6'
    reg = df6['region'].iloc[-1]
    df6 = df6.drop('region', axis=1)
    dataset = df6 
    dataset.name = str('df6_' + str(reg))


In [4]:
rootdir = 'zults/'
predictions = pd.DataFrame()
deviations = pd.DataFrame()
for file in tqdm(os.listdir(rootdir)):
    if 'predictions' in file:
        file_reader = pd.read_csv(rootdir + file)
        predictions = predictions.append(file_reader, ignore_index = True)
    
    if 'deviations' in file:
        
        file_reader = pd.read_csv(rootdir + file)
        deviations = deviations.append(file_reader, ignore_index = True)
        
deviations = deviations.drop('Unnamed: 0', axis = 1)

predictions = predictions.drop('Unnamed: 0', axis = 1)

deviations['total parameters'] = deviations['total parameters'].astype(int)
deviations['trained parameters'] = deviations['trained parameters'].astype(int)
deviations['total inputs'] = deviations['total inputs'].astype(int)


deviations = deviations[
    (deviations['df'] == 'df2') 
#     &
#     (deviations['layer architecture'] == '24-12')
#     &
#     (deviations['learning rate'] == 0.010)
#     &
#     (deviations['epochs'] != 100)

]

# deviations = deviations[deviations['epochs'] == ]

deviations = deviations.sort_values(
    [
#         'test mae avg', 
#         'test predicted thickness std dev'
        'layer architecture',
        'learning rate'
    ]
)
deviations

100%|██████████████████████████████████████████| 40/40 [00:00<00:00, 107.41it/s]


,layer architecture,total parameters,trained parameters,total inputs,df,dropout,learning rate,validation split,epochs,test mae avg,train mae avg,test mae std dev,train mae std dev,test predicted thickness std dev,train predicted thickness std dev
61,10-5,180,161,3834,df2,1,0.001,0.2,100,33.708867,28.946621,7.315369,7.274568,10.355960,10.696787
97,10-5,180,161,3834,df2,0,0.001,0.2,100,33.080288,28.433623,2.241167,2.065054,2.911484,3.326176
56,10-5,180,161,3834,df2,1,0.010,0.2,100,28.465352,21.216063,0.547645,0.346056,0.656038,0.644445
92,10-5,180,161,3834,df2,0,0.010,0.2,100,28.498946,21.287111,0.697538,0.343267,0.954383,1.117548
54,10-5,180,161,3834,df2,1,0.100,0.2,100,28.186652,21.048274,2.767640,2.805026,4.646422,5.144084
59,10-5,180,161,3834,df2,1,0.100,0.2,40,30.053667,22.517166,3.229234,3.306296,5.619617,6.280653
86,10-5,180,161,3834,df2,0,0.100,0.2,100,27.428805,20.532520,1.196918,1.084505,3.831624,4.151211
95,10-5,180,161,3834,df2,0,0.100,0.2,40,28.902645,22.339623,3.454076,3.414215,5.655355,6.578225
58,50-28,1976,1957,3834,df2,1,0.001,0.2,100,28.915888,22.607363,0.373358,0.180296,0.579332,0.436626
94,50-28,1976,1957,3834,df2,0,0.001,0.2,100,28.855653,22.634018,0.298559,0.193317,0.460799,0.377349


In [ ]:
"""
Here we load top rated model and look at predicted accuracies of each random state
"""
print('Please select index from deviations table to inspect further')

selection = int(input())

arch = deviations['layer architecture'].loc[selection]
top_learning_rate = deviations['learning rate'].loc[selection]
epochs = deviations['epochs'].loc[selection]
dropout = deviations['dropout'].loc[selection]
print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

dnn_model = {}
rootdir = 'saved_models/' + module + '/sm_' + arch + '/'


train_features, test_features, train_labels, test_labels = gl.data_splitter(dataset)
features = [train_features, test_features]
features = pd.concat(features)
labels = [train_labels, test_labels]
labels = pd.concat(labels)
for rs in tqdm(RS):
    
    hhh = (
        str(arch) + 
        '_' + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )   
    
    dnn_model[hhh] = tf.keras.models.load_model(
        rootdir + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    
    y = dnn_model[hhh].predict(features, verbose = 0)
    fig,ax=plt.subplots(1,1,figsize=(15,10))
    fig.patch.set_facecolor('w')
    plt.plot(labels,y,'.')
    plt.plot((0,400),(0,400),'-')
    plt.xlabel('True Thickness (m)')
    plt.ylabel('Model Thickness (m)')
    ax.set_title('Random State ' +str(rs))
    plt.xlim((0,400))
    plt.ylim((0,400))

Please select index from deviations table to inspect further


In [ ]:

fig_dir = (
    'figs/' + 
    module + 
    '/'   
) 
isdir = os.path.isdir(fig_dir)
if isdir == False:
    os.makedirs(fig_dir)

print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

print(fig_dir)
rootdir = 'saved_results/' + res + '/sr_' + arch + '/'
print(rootdir)
dnn_history = {}
for rs in RS: 
    
    history_name = (
        arch + 
        '_' +
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    model_name = (
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    dnn_history[model_name] = pd.read_csv(rootdir + model_name)

    fig,ax=plt.subplots(1,1,figsize=(10,10))
    fig.patch.set_facecolor('w')
    ax.set_title(history_name)
    gl.plot_loss(dnn_history[model_name])
    
file_name = (
    fig_dir +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) + '_' +
    str(rs) + '_' +
    '.eps'
)

print(file_name)
fig.savefig(file_name)

In [ ]:
ccccombo_breaker()
# past this point is under construction. 
# here be monsters

In [30]:
tau_b = 10*5
rho = 0.9167
g = 9.81

root_dir = 'zults/'
RGI_predicted = pd.DataFrame(columns = [
    'dataframe',
    'architecture',
    'learning rate',
    'epochs'
])

# read each prediction file and do stuff to it and append it to a table
for file in os.listdir(root_dir):
    if 'RGI_predicted' in file:
        file_reader = pd.read_csv(root_dir + file)
        
        file_reader['volume'] = (
            file_reader['avg predicted thickness'] / 1e3
        ) * file_reader['Area']
        
        # have to turn something into a series and append it to the df to build it
        sum_volume = sum(file_reader['volume'])
        total_volume = pd.Series(sum_volume, name = 'total volume')
        RGI_predicted = RGI_predicted.append(total_volume, ignore_index = True)
        
        # RGI_predicted.loc[
        #     RGI_predicted.index[-1], 'median predicted thickness'
        # ] = sum_3
        
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'sum(thickness) * sum(area)'
        ] = (sum(file_reader['avg predicted thickness']/1e3) * sum(file_reader['Area']))

        RGI_predicted.loc[
            RGI_predicted.index[-1], 'sum (area * thickness)'
        ] = sum_volume/1e3
        
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'volume mean'
        ] = file_reader['volume'].mean()
        
        
        # variance
        file_reader['variance'] = file_reader['predicted thickness std dev'] **2 
        variance = sum(file_reader['variance'])
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'total variance'
        ] = np.sqrt(variance)/1e3

        # mean thickness
        thickness_mean = file_reader['avg predicted thickness'].mean()
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'mean model thickness (km)'
        ] = thickness_mean/1e3
        
        # median thickness
        thickness_median = file_reader['avg predicted thickness'].median()
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'median model thickness (km)'
        ] = thickness_median/1e3
        
        # median distance from mean thickness
        thickness_median = file_reader['avg predicted thickness'].median()
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'median distance from mean'
        ] = abs(thickness_mean - thickness_median)/1e3
        
        # max thickness
        thickness_max = file_reader['avg predicted thickness'].max()
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'max model thickness (km)'
        ] = thickness_max/1e3
        
        # area * mean thickness
        area = sum(file_reader['Area'])
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'calculated global volume 10*3 km*3'
        ] = (area * (thickness_mean/1e3))/1e3
        
        # df1 has no lmax, so deriving thickness will not work
        if 'df1' not in file:
            file_reader['derived h max'] = (
                np.sqrt((2 * tau_b) / (rho * g)) * (file_reader['Lmax']/1e3)
            )

            file_reader['derived h avg'] = (
                (2/3) * file_reader['derived h max']
            )
            
            
            derived_h_max_sum = sum(file_reader['derived h max'])
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'derived h max (m)'
            ] = derived_h_max_sum/1e3
            
            
            derived_h_avg_sum = sum(file_reader['derived h avg'])
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'derived h avg (m)'
            ] = derived_h_avg_sum/1e3
            
        if 'df1' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df1'
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '16-8' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '16-8'
            if '24-12' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '24-12'
        if 'df2' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df2'   
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '50-28' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '50-28'
            if '64-48' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '64-48'
        if 'df3' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df3'        
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '37-20' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '37-20'
            if '59-28' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '59-28'
        if 'df4' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df4'    
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '47-21' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '47-21'
            if '64-36' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '64-36'
                
        if '0.1' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.100'
        if '0.01' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.010'
        if '0.001' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.001'
            
        if '_20' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '20'
        if '_25' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '25'
        if '_50' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '50'
        if '_60' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '60'
        if '_15' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '15'
        if '_30' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '30'
        if '_40' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '40'
        if '_100' in file:
            RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '100'

# RGI_predicted = RGI_predicted.drop(0, axis=1)
RGI_predicted.sort_values([
    # 'mean model thickness (km)',
    'architecture',
    'learning rate',
    'dataframe'
], ascending = True)

,dataframe,architecture,learning rate,epochs,0,sum(thickness) * sum(area),sum (area * thickness),volume mean,total variance,mean model thickness (km),median model thickness (km),median distance from mean,max model thickness (km),calculated global volume 10*3 km*3,derived h max (m),derived h avg (m)
6,df3,10-5,0.010,25,551131.245710,5.774722e+09,551.131246,2.645663,11.029310,0.045254,0.042424,0.002830,10.861602,27.721106,1013.52358,675.682386
17,df3,10-5,0.010,100,165101.206584,5.085758e+09,165.101207,0.792556,5.520567,0.039855,0.038784,0.001071,2.398133,24.413785,1013.52358,675.682386
4,df4,10-5,0.010,100,329423.235719,4.744391e+09,329.423236,1.581371,6.527242,0.037180,0.034355,0.002824,5.496152,22.775083,1013.52358,675.682386
9,df4,10-5,0.010,20,483541.597787,6.046576e+09,483.541598,2.321204,6.701245,0.047384,0.044090,0.003294,9.224206,29.026118,1013.52358,675.682386
10,df1,10-5,0.100,20,666249.807138,5.560925e+09,666.249807,3.198281,12.075867,0.043578,0.037613,0.005966,15.606449,26.694789,NaN,NaN
15,df1,10-5,0.100,100,473523.797837,5.409242e+09,473.523798,2.273114,18.031295,0.042390,0.037031,0.005358,10.971224,25.966645,NaN,NaN
0,df2,10-5,0.100,100,194176.608712,1.518147e+10,194.176609,0.932130,26.903187,0.118970,0.113275,0.005695,3.789173,72.877488,1013.52358,675.682386
19,df2,10-5,0.100,40,173749.981253,1.122855e+10,173.749981,0.834073,9.358447,0.087993,0.087824,0.000169,3.557514,53.901800,1013.52358,675.682386
13,df1,16-8,0.100,100,426019.401426,5.053780e+09,426.019401,2.045073,13.595510,0.039604,0.037350,0.002254,9.795820,24.260277,NaN,NaN
21,df1,16-8,0.100,20,590847.294312,5.708574e+09,590.847294,2.836317,9.555959,0.044735,0.038609,0.006127,13.834805,27.403569,NaN,NaN


In [ ]:
# """
# This cell contains code to produce histograms of all the architectures different histories
# """

# for i in deviations.index:
# #     print(i)
#     df = deviations.iloc[i]
#     arch = df['layer architecture']
#     top_learn_rate = df['learning rate']
#     epochs = df['epochs']

#     dfs = predictions[
#         (predictions['architecture'] == arch) &
#         (predictions['learning rate'] == top_learn_rate) &
#         (predictions['epochs'] == epochs)
#     ]
#     fig,ax = plt.subplots()
#     ax.set_title('df:' + str(i) + ' Layer architecture: ' + arch )
#     ax.set_ylabel('prediction count')
#     ax.set_xlabel('thickness (m)')
#     fig.patch.set_facecolor('w')
#     plt.hist(dfs['avg test thickness'])


In [ ]:
print('loading RGI...')
rootdir = '/data/fast0/datasets/rgi60-attribs/'
RGI_extra = pd.DataFrame()
for file in tqdm(os.listdir(rootdir)):
    f = pd.read_csv(rootdir+file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = RGI_extra.append(f, ignore_index = True)
    

RGI = RGI_extra[[
    'RGIId',
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax'
]]

df1 = RGI_predicted_df1
(sum((df1['area'] * df1['avg predicted thickness']) * 10**6) / 10**9)


AIC = deviations_df1['model parameters'] - deviations_df1['test mae avg']

AIC.sort_values()

fig,ax = plt.subplots()
fig.patch.set_facecolor('w')
plt.plot(
    deviations_df1['test predicted thickness std dev'], 
    deviations_df1['test mae avg'],
    '.',
)

loading RGI...


FileNotFoundError: [Errno 2] No such file or directory: '/data/fast0/datasets/rgi60-attribs/'

In [ ]:
"""
This is a density plot of the most recent predictions made. Can easily be modified in a loop
to show multiple random states and whatnot
"""
sns.set(rc={"figure.figsize":(15,10)})
sns.kdeplot(x = test_labels, y = y.flatten(),fill = True)
plt.plot((0,300),(0,300),'-')

In [ ]:
"""
This cell computes the true average thickness of the glaciers in use
"""


pth = '/data/fast1/glacierml/T_models/'
T_lab = pd.read_csv(pth + 'T.csv', low_memory = False)
T_lab = T_lab[[
    'GlaThiDa_ID',
    'LAT',
    'LON',
    'AREA',
    'MEAN_SLOPE',
    'MEAN_THICKNESS'
]]
T_lab = T_lab.dropna()

tru_thickness = np.sum(T_lab['MEAN_THICKNESS']) / len(T_lab['MEAN_THICKNESS'])
tru_thickness